# 第10章 注意力机制

## 10.1 注意力提示

### 练习10.1.1

在机器翻译中通过解码序列词元时，其自主性提示可能是什么？非自主性提示和感官输入又是什么？

**解答：**

### 练习10.1.2

随机生成一个$10 \times 10$矩阵并使用`softmax`运算来确保每行都是有效的概率分布，然后可视化输出注意力权重。

**解答：**

## 10.2 注意力汇聚：Nadaraya-Watson核回归

### 练习10.2.1

增加训练数据的样本数量，能否得到更好的非参数的Nadaraya-Watson核回归模型？

**解答：**

### 练习10.2.2

在带参数的注意力汇聚的实验中学习得到的参数$w$的价值是什么？为什么在可视化注意力权重时，它会使加权区域更加尖锐？

**解答：**

### 练习10.2.3

如何将超参数添加到非参数的Nadaraya-Watson核回归中以实现更好地预测结果？

**解答：**

### 练习10.2.4

为本节的核回归设计一个新的带参数的注意力汇聚模型。训练这个新模型并可视化其注意力权重。

**解答：**

## 10.3 注意力评分函数

### 练习10.3.1

修改小例子中的键，并且可视化注意力权重。可加性注意力和缩放的“点－积”注意力是否仍然产生相同的结果？为什么？

**解答：**

### 练习10.3.2

只使用矩阵乘法，能否为具有不同矢量长度的查询和键设计新的评分函数？

**解答：**

### 练习10.3.3

当查询和键具有相同的矢量长度时，矢量求和作为评分函数是否比“点－积”更好？为什么？

**解答：**

## 10.4 Bahdanau 注意力

### 练习10.4.1

在实验中用LSTM替换GRU。

**解答：**

### 练习10.4.2

修改实验以将加性注意力打分函数替换为缩放点积注意力，它如何影响训练效率？

**解答：**

## 10.5 多头注意力

### 练习10.5.1

分别可视化这个实验中的多个头的注意力权重。

**解答：**

### 练习10.5.2

假设有一个完成训练的基于多头注意力的模型，现在希望修剪最不重要的注意力头以提高预测速度。如何设计实验来衡量注意力头的重要性呢？

**解答：**

## 10.6 自注意力和位置编码

### 练习10.6.1

假设设计一个深度架构，通过堆叠基于位置编码的自注意力层来表示序列。可能会存在什么问题？

**解答：**

### 练习10.6.2

请设计一种可学习的位置编码方法。

**解答：**

## 10.7 Transformer

### 练习10.7.1

在实验中训练更深的Transformer将如何影响训练速度和翻译效果？

**解答：**

### 练习10.7.2

在Transformer中使用加性注意力取代缩放点积注意力是不是个好办法？为什么？

**解答：**

### 练习10.7.3

对于语言模型，应该使用Transformer的编码器还是解码器，或者两者都用？如何设计？

**解答：**

### 练习10.7.4

如果输入序列很长，Transformer会面临什么挑战？为什么？

**解答：**

### 练习10.7.5

如何提高Transformer的计算速度和内存使用效率？提示：可以参考论文 :cite:`Tay.Dehghani.Bahri.ea.2020`。

**解答：**

### 练习10.7.6

如果不使用卷积神经网络，如何设计基于Transformer模型的图像分类任务？提示：可以参考Vision Transformer。

**解答：**